In [3]:
import pandas as pd
import numpy as np

### Step 1  Loading the csv file.  

In [4]:
cols = ['bookID' ,'title',  'authors', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages', 'ratings_count',      
        'text_reviews_count', 'publication_date', 'publisher',  'coauthor']
df = pd.read_csv('books_data.csv', names=cols, skiprows=1,engine='python', quotechar=',', on_bad_lines='warn')

### Note: We have used quoting charcter ',' to prevent quoting issues. Also, engine is python. It is used when no of columns are variable.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bookID              11127 non-null  int64 
 1   title               11127 non-null  object
 2   authors             11127 non-null  object
 3   average_rating      11127 non-null  object
 4   isbn                11127 non-null  object
 5   isbn13              11127 non-null  object
 6   language_code       11127 non-null  object
 7   num_pages           11127 non-null  object
 8   ratings_count       11127 non-null  int64 
 9   text_reviews_count  11127 non-null  int64 
 10  publication_date    11127 non-null  object
 11  publisher           11127 non-null  object
 12  coauthor            4 non-null      object
dtypes: int64(3), object(10)
memory usage: 1.1+ MB


In [6]:
df.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,coauthor
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,None
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,None


###  Step 2 Remove dirty data for four rows.

#### As we can see the last column is problematic. Its is actually a co author name. But this co author name is present for only four rows in the datsset. 

#### Lets examine those four rows. 

In [7]:
dirty = df.loc[~df['coauthor'].isnull(), : ]

In [8]:
dirty

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,coauthor
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,Jr./Sam B. Warner,3.58,0674842111,9780674842113,en-US,236,61,6,4/20/2004,Harvard University Press
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,one of the founding members of this Tolkien w...,3.58,1593600119,9781593600112,eng,400,26,4,4/6/2004,Cold Spring Press\t
5877,22128,Patriots (The Coming Collapse),James Wesley,Rawles,3.63,156384155X,9781563841552,eng,342,38,4,1/15/1999,Huntington House Publishers
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,Son & Ferguson,0.00,0851742718,9780851742717,eng,49,0,0,5/1/1977,Brown Son & Ferguson Ltd.


#### The average rating for these four rows is actaully a author name.

In [9]:
coauthor = dirty.loc[:, 'average_rating']
coauthor = coauthor.rename("author2")

#### We will shift every other column to the left and we call this right side of data frame. 

In [10]:
right = dirty.loc[:, 'average_rating':'coauthor'].shift(periods=-1, axis=1)

#### All the other columns before author and taken as left. 

In [11]:
left = dirty.loc[:, :'authors']

#### We will merge the co author with author.  

In [13]:
left['authors'] = left['authors'] + coauthor

#### The left side will look like this. 

In [14]:
left

,bookID,title,authors
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner Jr./Sam B. Warner
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net one o...
5877,22128,Patriots (The Coming Collapse),James Wesley Rawles
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown Son & Ferguson


#### The Right side

In [15]:
right

,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,coauthor
3348,3.58,0674842111,9780674842113,en-US,236,61,6,4/20/2004,Harvard University Press,NaN
4702,3.58,1593600119,9781593600112,eng,400,26,4,4/6/2004,Cold Spring Press\t,NaN
5877,3.63,156384155X,9781563841552,eng,342,38,4,1/15/1999,Huntington House Publishers,NaN
8979,0.00,0851742718,9780851742717,eng,49,0,0,5/1/1977,Brown Son & Ferguson Ltd.,NaN


####  We will combine them. and call it clean. 

In [16]:
clean = pd.concat([left,  right, coauthor], axis=1)

In [17]:
clean = clean.drop(columns= ['coauthor'])

In [18]:
clean

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,author2
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner Jr./Sam B. Warner,3.58,0674842111,9780674842113,en-US,236,61,6,4/20/2004,Harvard University Press,Jr./Sam B. Warner
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net one o...,3.58,1593600119,9781593600112,eng,400,26,4,4/6/2004,Cold Spring Press\t,one of the founding members of this Tolkien w...
5877,22128,Patriots (The Coming Collapse),James Wesley Rawles,3.63,156384155X,9781563841552,eng,342,38,4,1/15/1999,Huntington House Publishers,Rawles
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown Son & Ferguson,0.00,0851742718,9780851742717,eng,49,0,0,5/1/1977,Brown Son & Ferguson Ltd.,Son & Ferguson


#### We will replace the dirty rows with this newly created clean data frame. 

In [19]:
df.loc[clean.index, :] = clean

### Step 3 We will remove 'coauthor', 'bookID', 'isbn', 'isbn13' columns as they do not make any sense wrt to rating prediction. 

In [20]:
df = df.drop(columns= ['coauthor', 'bookID', 'isbn', 'isbn13'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               11127 non-null  object
 1   authors             11127 non-null  object
 2   average_rating      11127 non-null  object
 3   language_code       11127 non-null  object
 4   num_pages           11127 non-null  object
 5   ratings_count       11127 non-null  int64 
 6   text_reviews_count  11127 non-null  object
 7   publication_date    11127 non-null  object
 8   publisher           11127 non-null  object
dtypes: int64(1), object(8)
memory usage: 782.5+ KB


### Step 4 We will also convert the numeric columns to int/ float values. 

In [22]:
df['average_rating'] = df['average_rating'].astype(np.float32)
df['num_pages'] = df['num_pages'].astype(np.int32)
df['ratings_count'] = df['ratings_count'].astype(np.int32)
df['text_reviews_count'] = df['text_reviews_count'].astype(np.int32)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               11127 non-null  object 
 1   authors             11127 non-null  object 
 2   average_rating      11127 non-null  float32
 3   language_code       11127 non-null  object 
 4   num_pages           11127 non-null  int32  
 5   ratings_count       11127 non-null  int32  
 6   text_reviews_count  11127 non-null  int32  
 7   publication_date    11127 non-null  object 
 8   publisher           11127 non-null  object 
dtypes: float32(1), int32(3), object(5)
memory usage: 608.6+ KB


#### Let's just check our plan worked or not! 

In [24]:
df.iloc[3348, :]

title                 Streetcar Suburbs: The Process of Growth in Bo...
authors                               Sam Bass Warner Jr./Sam B. Warner
average_rating                                                     3.58
language_code                                                     en-US
num_pages                                                           236
ratings_count                                                        61
text_reviews_count                                                    6
publication_date                                              4/20/2004
publisher                                      Harvard University Press
Name: 3348, dtype: object

### Step 4 It did Now let's convert dates 